<a href="https://colab.research.google.com/github/jessicamarycooper/Backwards/blob/main/Backwards_Matthew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

In [2]:
!pip install update transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel, utils
import torch
from matplotlib import pyplot as plt
%matplotlib inline
from IPython import display
import numpy as np
from tqdm import tqdm
from time import time
import json
utils.logging.set_verbosity_error()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vocab_len= 50257
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", padding_side='left')
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id, vocab_size=vocab_len).to(device)
model.eval()
# the model will be in evaluation, not training, mode throughout
word_embeddings = model.transformer.wte.weight.to(device)   
# 'word_embeddings' tensor gives emeddings for each token in the vocab for this model,
# has shape (vocab_len, embedding_dimension) which in this case = (50257, 768)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.4 MB/s eta 0:00:00


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [3]:
def normalise(x, min_max=[]):     
# normalises values of (array or tensor) x according to first (min) and second (max) values in list min_max. 
# This effectively defaults to [0,1] if the list doesn't contain exactly two elements. 
# The original code threw an error if min_max had length 1, so it's been changed slightly.

# First normalise x to [0,1]
    rnge = x.max() - x.min()
    if rnge > 0:
        x = (x - x.min())/rnge

# Now, if there's a min and max given in min_max list, multiply by difference and add minimum
    if len(min_max) > 1:
        rnge = min_max[1] - min_max[0]
        x = x * rnge + min_max[0]

    return x


def closest_tokens(emb, n=1):      
# This finds the n tokens in the vocabulary that are closest in the embedding space (in terms of Euclidean distance) to a given word embedding (‘emb’).
# Note that here 'emb' may or may not correspond to a token (i.e., it may or may not be a 'legal' embedding).
# Function returns a 4-tuple (list of the n tokens, list of their indices, list of their distances from emb, and list of their embedding vectors)
    torch.cuda.empty_cache()
    dists = torch.linalg.norm(word_embeddings - emb, dim=1)
    sorted_dists, ix = torch.sort(dists)	 
    # sorted_dists is a list of all embedding distances from 'emb', across entire vocab, sorted in increasing order, 
    # ix is a list of their corresponding 'vocab indices'
    tokens = [tokenizer.decode(i) for i in ix[:n]]
    # For each of the first n 'vocab indices' in ix, we decode it into the string version of the corresponding token. 
    # These strings then constitute the list 'tokens'.
    ixs = ix[:n]
    dists = sorted_dists[:n]
    embs = word_embeddings[ixs]  # Each of these n 'embeddings' is a tensor of shape (768,)
    return tokens, ixs, dists, embs  


def model_emb(inputs_embeds, output_len):
# 'input_embeds' is a tensor of shape (batch_size, input_len, embedding_dim)
# 'output_len' is an integer specifying the number of output tokens to generate
# Note that this function doesn't involve a target output. It simply takes a tensor of input embeddings (based on input length),
# calculates perplexities for that batch of input sequences,
# and runs the batch of input sequences through GPT2, for each finding next tokens iteratively 'output_len' number of times
    embs = inputs_embeds   # This is going to get expanded using 'output_embs'
    logits = []
    ixs = []
    input_logits = None
    for i in range(output_len):
        model_out = model(inputs_embeds=embs, return_dict=True)
        # Does a forward pass of GPT2 (or whichever model) on a batch of inputs (given as a tensor 'embs' of embeddings).
        # This 'embs' will expand along its 1st dimension with each iteration.
        # Outputs logits and more (hidden states, attention, etc.) as a dictionary 'model_out'.
        # But we'll only be concerned with model_out.logits.

        if i == 0:
            input_logits = model_out.logits 
            # On first pass through loop, we simply use the logits of the model output
            # That's a tensor of shape (batch_size, input_len, vocab_size) giving logits for each input in each batch.
            # Presumably for each input, this is conditioned on the inputs that preceded it?

        # On every pass throught the loop (including the first), we defined this tensor of shape (batch_size, 1, vocab_size):
        last_logits = model_out.logits[:,-1].unsqueeze(1)  
        # model_out.logits[:,-1] will be a 2D tensor of shape (batch_size, vocab_size), just giving logits for last input/embedding across all batches/tokens
        # unsqueezing, we get tensor of shape (batch_size, 1, vocab_size) also giving logits of last input/embedding, differently formatted  
        logits.append(last_logits)  # appends last_logits tensor to the 'logits' list 
        ix = torch.argmax(last_logits, dim=-1)  # for each batch, finds the vocab index of the token with the largest logit in last_logits
        ixs.append(ix) # ...and appends this tensor of shape (batch_size,) (containing indices) it to the list 'ixs'
        output_embs = word_embeddings[ix]   # for each batch, finds embedding for the token with that index...
        embs = torch.cat([embs, output_embs], dim=1)  #...concatenates that tensor of embeddings to the 'embs' tensor in the first dimension before next iteration

     # When the loop is completed 'embs' will be a tensor containing all of the input and output word embeddings produced by the model   
     # ...so presumably of shape (batch_size, input_len + output_len, embedding_dim)

    logits = torch.cat(logits, dim=1)   # this converts logits from a list of tensors to a single tensor, by concatenating all of the tensors in the list
                                        # it will have shape (batch_size, output_len, vocab_size)
    perp = perplexity(input_logits)     # 'input_logits' was calculated on first pass through loop where only input embeddings were involved
    return logits, embs, perp          
    # logits has shape (batch_size, output_len, vocab_size),         CHECK THAT!
    # embs has shape (batch_size, input_len + output_len, embedding_dim)
    # perp has shape (batch_size,)


def perplexity(logits):
    # logits is of shape (batch_size, 'sequence length', vocab_size)
    # for all current calls, 'sequence length' is going to be input_len
    probs, ix = torch.max(torch.softmax(logits, dim=-1), dim=-1)
    # torch.softmax(logits, dim=-1) will also be a tensor of shape (batch_size, 'sequence length', vocab_size), 
    # but where the logits in the last dimension get converted into probabilities via softmax. torch.max() then pull out the largest of these and its index
    # probs is a tensor that contains the maximum probability for each token in the embedding sequence, shape (batch_size, 'sequence length')
    # ix is a tensor that contains the corresponding indices, also with shape (batch_size, 'sequence length')
    perp = 1/ (torch.prod(probs, dim=-1)**(1/probs.shape[-1])) - 1
    # defines a scalar that's larger with greater uncertainty (so if the probs are small, their product is small, the reciprocal of some power is large)
    # probs.shape[-1] is output_len; the idea of raising the probs product to power 1/output_len is to make perplexities comparable across different output lengths
    return perp


# Here's the key function that optimises for a sequence of input embeddings, given a target_output string:
def optimise_input(epochs=100, 
                   lr=0.1, 
                   rand_after=False,    # Do we re-initialise inputs tensor with random entries when an optimal input is found?
                   w_freq=10,           # logging (write) frequency
                   base_input=False,      # If False, start_inputs will be entirely random
                   batch_size=1, 
                   input_len=1, 
                   target_output=tokenizer.eos_token,    # Default target output is the "end-of-string" token; this won't generally be used
                   output_len=None,
                   dist_reg=1,       # distance regularisation coefficient
                   perp_reg=0,       # perplexity regularisation coefficient; setting to 0 means perplexity loss isn't a thing
                   plt_loss=False,   # Do we plot loss?
                   loss_type='log_prob_loss', 
                   seed=0,
                   return_early=True,    # finishes if single optimised input is found
                   verbose=0,            # Controls how much info gets logged.
                   lr_decay=False,       # Use learning rate decay? If so, a scheduler gets invoked.
                   noise_coeff = 0.01, **kwargs):     # Introduced for generality in the construction of start_input[1:] below.
    
    torch.manual_seed(seed)               # sets up PyTorch random number generator

    results = {'args': locals()}

    if plt_loss:
        plt.rcParams.update({'figure.figsize': (40,6)})

    total_losses = []
    losses = []
    dists = []
    perps = []
    optimised_inputs = dict()
    done = None

    output_ix = tokenizer.encode(target_output, return_tensors='pt')[0].to(device)
    # output_ix is a 1-D tensor of shape (output_len,) that contains the indices of the tokens in the encoding of the string 'target_output'
    # tokenizer.encode(target_output, return_tensors='pt') is a list containing this one tensor, hence the need for the [0]
    # "return_tensors='pt'" ensures that we get a tensor in PyTorch format

    if output_len == None or output_len < output_ix.shape[0]:                    # This won't generally be the case, but if we don't specify output_len (i.e. it's == None), then...
        output_len = output_ix.shape[0]       # ...it will be set to the number of tokens in the encoding of the string 'target_output'
    # Why not just set output_len = output_ix.shape[0] in all cases?
    # Will there be situations where we want output_len to be of a different size to the number of tokens in target_output?

    print('Optimising input of length {} to maximise output logits for "{}"'.format(input_len, target_output))
    # Typically this would print something like 'Optimising input of length 6 to maximise output logits for "KILL ALL HUMANS!"'.

    if base_input == False:
        start_input = torch.rand(batch_size, input_len, word_embeddings.shape[-1]).to(device)
        # If no base_input is provided, we construct start_input as a random tensor 
        # of shape (batch_size, input_len, embedding_dim) (embedding_dim = 768 for this GPT-2 model).
        start_input = normalise(start_input,[word_embeddings.min(dim=0)[0], word_embeddings.max(dim=0)[0]])
        # We normalise this random tensor so that its minimum and maximum values correspond to those in the entire word_embeddings tensor
        # This dispenses with whole swathes of "input space" which contain no legal token embeddings 
        # (we're limiting ourselves to a kind of "hull" defined by the 50527 vocab tokens in the embedding space), 
        # which is a sensible place to look for optimised inputs.
    else:
        start_input = word_embeddings[output_ix].mean(dim=0).repeat(batch_size, input_len, 1)

        if batch_size > 1:
            start_input[1:] += (torch.rand_like(start_input[1:]) + torch.full_like(start_input[1:], -0.5)) * noise_coeff
        #...and if we have more than one element in our batch, we "noise" the rest. 
        # This was originally done using "*=" (multiplying entries by small random numbers)
        # We've changed this to "+=" (adding  small random numbers instead of multiplying by them).
        # The original code would have pushed everything in a positive direction, hence the use of a tensor full of -0.5's.       

    
    input = torch.nn.Parameter(start_input, requires_grad=True)
    # input is not a tensor, it's a Parameter object that wraps a tensor and adds additional functionality. 
    # 'input.data' is used below
    
    optimiser = torch.optim.Adam([input], lr=lr)
    # standard optimiser; note that it generally operates on a list of tensors, so we're giving it a list of one tensor; standard learning rate
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', patience=20, cooldown=20, factor=0.5)
    # this is used when loss hasn't improved for 20 timesteps; this scheduler will reduce the lr by a 'factor' of 0.5 when the 
    # validation loss stops improving for 'patience' (here 20) epochs, and will wait 'cooldown' (here 20) epochs before resuming normal operation.

    # now we loop across training epochs
    for e in range(epochs):

        logits, emb, perp = model_emb(torch.clamp(input, word_embeddings.min(), word_embeddings.max()), output_len)
        # Does forward pass on a 'clamped' version of the 'input' tensor (done to contain it within the 'hull' of the vocabulary within 'input space').
        # Iterates to produce an output of output_len tokens, 
        # returns: 'logits' = tensor of logits for output, of shape (batch_size, output_len, vocab_size)
        # 'emb': tensor of embeddings for input+output of shape (batch_size, input_len + output_len, embedding_dim); 
        # 'perp': the input sequence perplexities tensor, of shape (batch_size,)
        probs = torch.softmax(logits, dim=-1)
        # For each batch, output, converts the sequence of logits (of length 'vocab_size') in the 'logits' tensor to probabilities, using softmax

        logits = (logits - logits.min(dim=-1)[0].unsqueeze(-1)) / (logits.max(dim=-1)[0].unsqueeze(-1) - logits.min(dim=-1)[0].unsqueeze(-1))
        # This appears to be normalising the logits for each batch/output embedding so they're all between 0 and 1... 
        # This is for ease of visualisation.

        perp_loss = perp.mean() * perp_reg
        # That's taking the mean perp value across all batches, then regularising it. Currently perp_reg is set to 0, so perp_loss = 0.

        if output_len > output_ix.shape[0]:
            target_logits = torch.stack([logits[:, :, ix] for ix in output_ix], dim=-1)
            target_logits = torch.max(target_logits, dim=-1)[0]
            # logits is shape (batch_size, output_len, vocab_size) 
            # We throw out everything in the final dimension except those logits corresponding to indices of tokens in the target_ouput
            # This gives tensor with shape (batch_size, output_len, output_ix.shape[0])
            # We then take the maximum of those for each batch, output; this gives shape (batch_size, output_len)
            # The [0] returns just the max (torch.max returns (max, indices) tuple)
            target_probs = torch.stack([probs[:, :, ix] for ix in output_ix], dim=-1)
            target_probs = torch.max(target_probs, dim=-1)[0]
            # This does the analogous thing for probs.

        else:
            target_logits = torch.stack([logits[:,i, ix] for i, ix in enumerate(output_ix)], dim=-1)
            target_probs = torch.stack([probs[:,i, ix] for i, ix in enumerate(output_ix)], dim=-1)
            # This handles case where output_len == output_ix.shape[0]
            # target_logits now of shape (batch_size, output_len)?
            # output_len < output_ix.shape[0] was dealt with in line 135
            
        token_dist = torch.stack([torch.stack([closest_tokens(e)[2].squeeze(-1) for e in input[b]]) for b in range(batch_size)])
        # As far as I can tell, this creates a tensor of shape (batch_size, input_len, 1) which gives distance to nearest
        # legal token embedding for each input embedding in each batch
        mean_token_dist = token_dist.mean() * dist_reg
        # A single scalar value, taking mean across the batch and input embeddings? 


        # There are currently four loss types, many more could be introduced.
        # log_prob_loss is the current default.
        if loss_type == 'logit_loss':
            loss = 1-target_logits
        elif loss_type == 'log_prob_loss':
            loss = -torch.log(target_probs)
        elif loss_type == 'prob_loss':
            loss = 1-target_probs
        elif loss_type == 'CE':
            loss = torch.nn.functional.cross_entropy(logits.swapaxes(-1,-2), output_ix.repeat(batch_size, 1), reduction=None)
        else:
            print(loss_type + 'is not implemented.')
            return 

        batch_loss = loss.mean()

        total_loss = torch.stack([mean_token_dist, batch_loss, perp_loss]).mean()
        # This is this just (mean_token_dist + loss + perp_loss)/3 tensorised across batches, yes?

        total_losses.append(total_loss.detach().cpu().data)
        losses.append(batch_loss.detach().cpu().data)
        dists.append(mean_token_dist.detach().cpu().data)
        perps.append(perp_loss.detach().cpu().data)
        # these four lists were intialised above. We're appeneding to the list each epoch. All are scalars.

        closest_ix = torch.stack([torch.stack([closest_tokens(e)[1] for e in b]) for b in input]).squeeze(-1)
        # This is similar to above, but building a tensor of indices of nearest embeddings, rather than distances.
        # Iterates over batches, and for each batch iterates over embeddings, giving tensor of shape (batch_size, input_len).

        model_outs = model.generate(closest_ix, max_length = output_len+input_len)
        # The 'closest_ix' tensor is passed as the initial input sequence to the model, 
        # and the max_length parameter specifies the maximum length of the total sequence to generate.
        # The output sequence will be terminated either when the end-of-sequence token is generated 
        # or when the maximum length is reached, whichever occurs first.
        # 
        # The output of the model.generate method will be a tuple containing the generated sequences and the model's internal states. 
        # The generated sequences will be stored in a tensor of shape (batch_size, output_len+input_len). 
        # Each element of the tensor will be a sequence of tokens with a length of at most output_len+input_len.
        
        for b in range(batch_size):

            if output_len > output_ix.shape[0]:
                if target_output in tokenizer.decode(model_outs[b][input_len:]):
                    done = tokenizer.decode(model_outs[b][:input_len])
                    optimised_inputs.update({done:loss[b].detach().cpu().numpy().tolist()})
                # model_outs[b][input_len:], for a batch b, is only looking at the *output* embeddings 
                # we decode these as tokens... is the target_output a substring?
                # if so, we print the target_output and the decoded string that contains it
                # 'done' is the string version of the model's output for given input, we add this to set 'optimised_inputs'.


            if tokenizer.decode(model_outs[b][input_len:]) == target_output:
                done = tokenizer.decode(model_outs[b][:input_len])
                optimised_inputs.update({done:loss[b].detach().cpu().numpy().tolist()})
                # model_outs[b][input_len:], for a batch b, is only looking at the *output* embeddings 
                # we decode these as tokens... is the target_output equal to output string?
                # Nothing printed in this case.
                # 'done' is the string version of the model's output for given input, we add this to set 'optimised_inputs'.
            
            if done is not None and rand_after:
                input.data[b] = torch.rand_like(input[b])
                # Random re-initialisation (if 'rand_after' set to True)

  
        if ((e+1) % w_freq == 0) or done and return_early:
            display.clear_output(wait=True) 
            print('\033[H\033[J')  
        # Every w epochs we write to log, unless we have found an optimised input before that and 'return_early' == True. 
        # I'm still not entirely sure about the idea of 'return_early'.

            if plt_loss:
                plt.plot(range(len(total_losses)), total_losses, label='Total Loss', color='black')
                plt.plot(range(len(losses)), losses, label='Output Loss')
                plt.plot(range(len(dists)), dists, label='Emb Dist Loss')
                plt.plot(range(len(perps)), perps, label='Perp Loss')
                plt.yscale('log')
                plt.legend()

                plt.show()

            print('Inputs found: ', optimised_inputs)
            print('{}/{} Output Loss: {} Emb Dist Loss: {} Perp Loss: {} LR: {}'.format(e+1, epochs, batch_loss, mean_token_dist, perp_loss, optimiser.param_groups[0]['lr']))
            if verbose == 3:
                print('Target Probs: {}\nTarget Logits: {}\nInput Dists: {}\nInput Perplexity: {}\n'.format(target_probs.detach().cpu().numpy(), target_logits.detach().cpu().numpy(), token_dist.detach().cpu().numpy(), perp.detach().reshape(-1).cpu().numpy()))
            # Optimised inputs and additional information are printed as part of log

            for b in range(batch_size):
                if verbose > 0:
                    if verbose == 2:
                        print(b, repr(' Raw embeddings: {}'.format(''.join([closest_tokens(e)[0][0] for e in emb[b]]))))
                        # Change name to clarify (output of model if we just put in raw embeddings)
                        # prints batch number; closest_tokens(e)[0] is a list of tokens, closest_tokens(e)[0] is the first (closest) of these
                        # these get joined with separator '' (SHOULDN'T THAT BE ' '?)  
                    print(b, repr(' Closest embeddings: {}'.format(tokenizer.decode(model_outs[b]), '\n')))
                        # WON'T THIS give string decodings of the embeddings, rather than the embeddings themselves?
                else:
                    print(repr(tokenizer.decode(model_outs[b])), end=' ')
                    # The least verbose printed output. The 'end' parameter is used to specify the end-of-line string that is appended to the output. 
                    # By default, this is a newline character, but in this case it has been set to a single space character, 
                    # so the output will be separated by spaces rather than newlines.

            if done and return_early:
                print('\nOptimised Input: "{}"'.format(done))
                results['optimised_inputs'] = optimised_inputs
                return results
                # we know optimised_inputs set contains a single element in this case
            
        optimiser.zero_grad()
        total_loss.backward()
        optimiser.step()
        # I assume these three lines are standard NN optimisation stuff?

        if lr_decay:
            scheduler.step(total_loss)
         # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', patience=20, cooldown=20, factor=0.5) gets used if lr_decay == True
        done = None

    results['optimised_inputs'] = optimised_inputs
    return results
    # that's a set of strings



In [71]:
# attempt to implement k-means algorithm

def kmeans(num_clusters):

    empty_cluster = True

    # euclidean distance threshold to break out of loop
    threshold = 0.01

    while empty_cluster == True: 
        # randomly generate num_clusters centroids, stack these into a tensor of shape (num_clusters, 768)
        # then normalise to vocab embedding span
        centroids = torch.rand(num_clusters, word_embeddings.shape[-1]).to(device)
        centroids = normalise(centroids,[word_embeddings.min(dim=0)[0], word_embeddings.max(dim=0)[0]])

        distances = torch.cdist(word_embeddings, centroids, p=2)
        # This will be of shape (vocab_len, num_clusters), recording distances of token embeddings from each of the centroid.
        closest_distance, closest_centroid = torch.min(distances, dim = -1)
        # These will be of shape (vocab_len,), recording the distance of each token embedding to nearest centroid, and the index of that centroid.

        clusters = []
        mean_distances = []
        # We iterate over the centroids
        for i in range(centroids.shape[0]):
            mask = closest_centroid == i  # This builds a Boolean mask over the complete set of tokens, True if a token's nearest centroid is the ith.
            clusters.append(word_embeddings[mask]) 
            # word_embeddings[mask] is a subtensor of the (50257, 768) shape tensor involving only tokens nearest ith centroid.
            # This subtensor gets appended to the list 'clusters', which has num_clusters elements.

        shapes = [clusters[i].shape[0] for i in range(len(clusters))]  
        if 0 not in shapes:
            empty_cluster = False 

    # Now we have a set on num_clusters non-empty clusters, we begin iterating centroid positions:
    centroids_stable = False
    iterations = 0

    while centroids_stable == False:
        iterations += 1
        distances = torch.cdist(word_embeddings, centroids, p=2)
        # This will be of shape (vocab_len, num_clusters), recording distances of token embeddings from each of the centroid.
        closest_distance, closest_centroid = torch.min(distances, dim = -1)
        # These will be of shape (vocab_len,), recording the distance of each token embedding to nearest centroid, and the index of that centroid.

        clusters = []
        mean_distances = []
        # We iterate over the centroids
        for i in range(centroids.shape[0]):
            mask = closest_centroid == i  # This builds a Boolean mask over the complete set of tokens, True if a token's nearest centroid is the ith.
            clusters.append(word_embeddings[mask]) 
            # word_embeddings[mask] is a subtensor of the (50257, 768) shape tensor involving only tokens nearest ith centroid.
            # This subtensor gets appended to the list 'clusters' (num_clusters elements).
            mean_distances.append(closest_distance[mask].mean().item()) 
            # closest_distance[mask] is a subtensor of the (50257,) shape tensor involving only tokens nearest ith centroid.
            # This subtensor gets appended to the list 'mean_distances' (num_clusters elements).

        new_centroids = []
        for i in range(num_clusters):
            new_centroids.append(clusters[i].mean(dim=0))
            # clusters[i].mean(dim=0) is the centroid of the set of vectors encoded in the tensor clusters[i]
            # these all get put in a list...

        new_centroids = torch.stack(new_centroids)
        # ... and the list gets stacked into a tensor of shape (num_clusters, 768)

        # We now compute the euclidean distance between old and new centroids
        distance = torch.norm(new_centroids - centroids, dim=-1)
        #print('iteration: ', iterations, '\n max distance between old and new centroids:', torch.max(distance).item())
        # This is returning a "nan" for all iterations in some cases, usually when num_clusters > 10
        # I can't figure out why this is happening, but possibly some error handling could just take us back 
        # to the random initialisation of the 'centroids' tensor.

        if torch.max(distance) < threshold:
            centroids_stable == True
            break
        centroids = new_centroids # if we're still outside the distance threshold, keep iterating
    
 
    #for i in range(len(clusters)):
        #print('Cluster', i, ' contains ', len(clusters[i]), ' embeddings.')
    #for j in range(num_clusters):
        #print('Closest token to centroid ', j,': ', closest_tokens(centroids[j])[0], closest_tokens(centroids[j])[1].item())
    return centroids

In [ ]:
# This runs 200 batches of 50 and keeps track of the most common closest tokens to centroid embeddings, and how many appeareances they make
token_counts = torch.zeros(vocab_len)
for j in range(200):
    print('batch', j)
    centroids = kmeans(50)
    for i in range(50):
        token_counts[closest_tokens(centroids[i])[1].item()] +=1

values, indices = torch.sort(token_counts, descending=True)
print(indices[:50], values[:50])

batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79
batch 80
batch 81
batch 82


In [64]:
most_common_tokens_idxs = [30212, 187, 195, 216, 182, 179, 213, 39820, 124, 199,
208, 125, 23090, 554, 30208, 3607, 281, 7003, 37528, 15524,
192, 42089,  217, 39752, 183, 210, 201, 209, 190, 287,
189, 211, 818, 206, 212, 2890, 1315, 203, 207, 14827,
30898, 218, 219, 28124, 215, 15563, 36173, 3179, 4035, 2670]
most_common_tokens = [tokenizer.decode(i) for i in most_common_tokens_idxs]
print(most_common_tokens)

[' externalToEVA', '�', '\x07', '\x1c', '�', '�', '\x19', '龍�', '�', '\x0b', '\x14', '�', 'ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ', ' In', ' externalTo', ' gives', ' an', 'Although', ' 258', ' 1978', '\x04', ' TheNitrome', '\x1d', 'quickShip', '�', '\x16', '\r', '\x15', '\x02', ' in', '\x01', '\x17', 'In', '\x12', '\x18', 'izing', ' 15', '\x0f', '\x13', 'ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ', 'embedreportprint', '\x1e', '\x1f', 'laughs', '\x1b', ' fork', ' RandomRedditor', 'nces', 'ifying', '39']


In [ ]:
ix = tokenizer.encode("")
# list of 'vocab indices'
print(ix)
print([tokenizer.decode(i) for i in ix])
# prints reconstruction of input string
print(len(ix))
# prints number of tokens
output_len=2
model_out = model.generate(torch.tensor(ix).unsqueeze(0).to(device), max_length = output_len + len(ix))
print(tokenizer.decode(model_out[0]))
# pushes input string throught GPT2 (or whichever model) iteratively producing output_len number of tokens, then prints input + output.

In [ ]:
experiments = [{'base_input': False, 
                'plt_loss': False, 
                'verbose': 1, 
                'epochs': 1000, 
                'lr_decay': False, 
                'return_early': False, 
                'lr': 0.1, 
                'batch_size': 20, 
                'target_output': ' a lot of data', 
                'output_len': 4, 
                'input_len': 3, 
                'w_freq': 20, 
                'dist_reg': 1, 
                'perp_reg': 0, 
                'loss_type': 'log_prob_loss',
                'note':''}
                ]


experiment_log = {}

In [ ]:

for e in experiments:
    tick = time()
    results = optimise_input(**e)
    tock = time()
    rt = tock - tick
    results.update({'runtime':rt})
    
    with open("backwards_results.json","a") as f:
        f.write(json.dumps({tick:results}))
